In [1]:
import sys, os
repo_start = f'../../'
sys.path.append(repo_start)

from modules.utils.imports import *
from modules.symbolic_net.model_wrapper import model_wrapper
from modules.symbolic_net.build_symbolic_net import symbolic_net
from modules.utils.training_test_split import training_test_split
from modules.analysis.generate_loss_curves import generate_loss_curves
from modules.loaders.format_data import format_data_general
from modules.generate_data.simulate_system import reaction
from modules.symbolic_net.write_terms import write_terms
from modules.symbolic_net.individual import individual
from modules.symbolic_net.visualize_surface import visualize_surface, visualize_surface
from modules.genetic_algorithm.custom_deap_functions import (
    calculate_poly_terms, calculate_hill_terms)

In [2]:
dir_name = '/work/users/s/m/smyersn/elston/projects/kinetics_binns/development/equation_fitting/symbolic_net/runs/param_bounds_symbolic_net_l1_reg_0.1_repeat_20_lr_0.001'
device = 'cpu'

In [3]:
config = {}
exec(Path(f'{dir_name}/config.cfg').read_text(encoding="utf8"), {}, config)

training_data_path = config['training_data_path']
species = int(config['species'])
degree = int(config['degree'])
nonzero_term_reg = int(config['nonzero_term_reg'])
l1_reg = float(config['l1_reg'])
param_bounds = float(config['param_bounds'])
density_weight = float(config['density_weight'])
learning_rate = float(config['learning_rate'])

# Set training hyperparameters
epochs = int(1e6)
rel_save_thresh = 0.05

# Generate training data
training_data = format_data_general(2, 2, file=training_data_path)
uv = training_data[:, -2:]
u_triangle_mesh, v_triangle_mesh = lltriangle(uv[:, 0], uv[:, 1])
u, v = np.ravel(u_triangle_mesh), np.ravel(v_triangle_mesh)

a, b, k = 1, 1, 0.01
F_true = reaction(u, v, a, b, k)

training_data_nans = np.stack((u, v, F_true), axis=1)

# Remove nans from training data
mask = ~np.isnan(training_data_nans).any(axis=1)
training_data = training_data_nans[mask]

# Split training data
device = 'cpu'
x_train, y_train, x_val, y_val = training_test_split(training_data, 1, device)

# initialize model and compile
sym_net = symbolic_net(species, degree, param_bounds, device, l1_reg,
                       nonzero_term_reg)
sym_net.to(device)

# initialize optimizer
parameters = sym_net.parameters()
opt = torch.optim.Adam(parameters, lr=learning_rate)

model = model_wrapper(
    model=sym_net,
    optimizer=opt,
    loss=sym_net.loss,
    augmentation=None,
    save_name=f'{dir_name}/binn')

In [4]:
training_data[:, -1]

array([-4.43099989e-06, -2.44513649e-02, -4.86528321e-02, ...,
       -4.43098383e-06, -2.39573902e-02, -4.43098380e-06])

In [15]:
def fix_insignificant_terms(ind, training_data):
    # removes all terms from individual that have minor impact on total
    # surface shape and magnitude
    term_surfaces = ind.predict_f(training_data.clone().detach(), terms=True)

    for i in range(len(ind.coeffs)):
        term_surface = term_surfaces[:, i]
        if (term_surface.max() - term_surface.min()) < 2:
            ind.coeffs[i] = 0
            
    ind.update_attributes(torch.cat((ind.coeffs, ind.hill_ks, ind.hill_ns)))
            
    # set k and n values for insignificant Hill functions to zero
    mask = ind.hill_params == 0
    ind.hill_ks = ind.hill_ks.masked_fill(mask, 0)
    ind.hill_ns = ind.hill_ns.masked_fill(mask, 0)
    
    ind.update_attributes(torch.cat((ind.coeffs, ind.hill_ks, ind.hill_ns)))


In [16]:
# load model    
weights = torch.load(f"{dir_name}/binn_best_val_model", map_location=device)
sym_net.load_state_dict(weights)

ind = individual(sym_net.params, species, degree)

print(f'Original Equation:')
terms = ind.write_terms()
for term in terms:
    print(f'{term}')

print(f'\n No insignificant terms')
fix_insignificant_terms(ind, torch.tensor(training_data))
# ind.fix_insignificant_terms(torch.tensor(training_data))
terms = ind.write_terms()
for term in terms:
    print(f'{term}')


print(f'\n No cheating Hill functions')
ind.fix_cheating_hill_functions(torch.tensor(training_data))
terms = ind.write_terms()
for term in terms:
    print(f'{term}')

Original Equation:
-0.902 * u
0.000 * v
-0.010 * u * u
0.001 * u * v
0.001 * v * v
-0.001 * u^0.000 / (1 + 0.000 * u^0.000)
-0.001 * v^0.000 / (1 + 0.000 * v^0.000)
-0.004 * u * v^0.000 / (1 + 0.000 * v^0.000)
0.908 * v * u^2.038 / (1 + 0.009 * u^2.038)
0.000 * (1 - u^0.022 / (1 + 0.000 * u^0.022))
0.000 * (1 - v^0.000 / (1 + 0.000 * v^0.000))
-0.000 * u * (1 - v^0.000 / (1 + 0.000 * v^0.000))
0.000 * v * (1 - u^0.000 / (1 + 0.000 * u^0.000))

 No insignificant terms
-0.902 * u
0.908 * v * u^2.038 / (1 + 0.009 * u^2.038)

 No cheating Hill functions
-0.902 * u
0.908 * v * u^2.038 / (1 + 0.009 * u^2.038)


In [6]:
def abic(individual, true_vals, predicted_vals):   
    # Calculate the residual sum of squares (RSS)   
    residuals = true_vals - predicted_vals
    rss = torch.sum(residuals ** 2)

    # Number of data points
    n = len(predicted_vals)

    # Estimate the variance of the residuals
    sigma_squared = rss / n

    # Calculate the log-likelihood
    log_likelihood = -n / 2 * (torch.log(2 * np.pi * sigma_squared) + 1)

    # Number of parameters (k)
    k = len(individual.params)

    # Calculate AIC
    aic = 2 * k - 2 * log_likelihood

    # Calculate BIC
    bic = k * np.log(n) - 2 * log_likelihood
    
    return aic, bic

In [7]:
weights = torch.load(f"{dir_path}/binn_best_val_model", map_location=device)
sym_net.load_state_dict(weights)

ind = individual(sym_net.params, poly_terms, hill_terms)
print(ind.params)

NameError: name 'dir_path' is not defined

In [7]:
individuals = []

# Specify the directory path
parent_dir = '/work/users/s/m/smyersn/elston/projects/kinetics_binns/development/equation_fitting/symbolic_net/runs/repeat_9_50k_stop_l1_0.1_repeats'

# List all directories in the given directory
child_dirs = [d for d in os.listdir(parent_dir) if os.path.isdir(os.path.join(parent_dir, d))]

for child_dir in child_dirs:
    
    dir_path = f'{parent_dir}/{child_dir}'

    # load model    
    model.load(f"{dir_path}/binn_best_val_model", device=device) 
    ind = individual(model.model.params, poly_terms, hill_terms)
     
    fn = f'{dir_path}/equation_refined.txt'
    file = open(fn, 'w')
   
    file.write(f'Original Equation:\n')
    terms = write_terms(ind)
    for term in terms:
        file.write(f'{term}\n')

    file.write(f'\nNo insignificant terms:\n')
    ind.fix_insignificant_terms(torch.tensor(training_data))
    terms = ind.write_terms()
    for term in terms:
        file.write(f'{term}\n')

    file.write(f'\nNo cheating Hill functions:\n')
    ind.fix_cheating_hill_functions(torch.tensor(training_data))
    terms = ind.write_terms()
    for term in terms:
        file.write(f'{term}\n')
    
    file.close()
    
    true_vals = torch.from_numpy(training_data[:, -1])
    predicted_vals = ind.predict_f(torch.from_numpy(training_data[:, :-1]))
    
    aic, bic = abic(ind, true_vals, predicted_vals)
    
    individuals.append((child_dir, ind.params, aic, bic))

In [12]:
individuals_sorted = sorted(individuals, key=lambda x: x[2])

for i in range(len(individuals_sorted)):
    # load model   
    ind = individual(individuals_sorted[i][1], poly_terms, hill_terms)
    print(individuals_sorted[i][0])
    
    terms = ind.write_terms()
    for term in terms:
        print(f'{term}')
    print('\n')


param_bounds_symbolic_net_l1_reg_0.1_repeat_22_lr_0.001
-0.983 * u
0.964 * v * u^2.021 / (1 + 0.010 * u^2.021)


param_bounds_symbolic_net_l1_reg_0.1_repeat_39_lr_0.001
-0.996 * u
0.980 * v * u^2.010 / (1 + 0.010 * u^2.010)


param_bounds_symbolic_net_l1_reg_0.1_repeat_37_lr_0.001
-0.917 * u
-0.010 * u * u
0.922 * v * u^2.031 / (1 + 0.009 * u^2.031)


param_bounds_symbolic_net_l1_reg_0.1_repeat_5_lr_0.001
-0.902 * u
-0.010 * u * u
0.908 * v * u^2.038 / (1 + 0.009 * u^2.038)


param_bounds_symbolic_net_l1_reg_0.1_repeat_46_lr_0.001
-0.925 * u
-0.010 * u * u
0.925 * v * u^2.029 / (1 + 0.009 * u^2.029)


param_bounds_symbolic_net_l1_reg_0.1_repeat_9_lr_0.001
-1.024 * u
1.143 * v * u^1.902 / (1 + 0.010 * u^1.902)


param_bounds_symbolic_net_l1_reg_0.1_repeat_3_lr_0.001
-0.007 * u * u
-1.231 * u * (1 - v^0.564 / (1 + 0.005 * v^0.564))
-0.565 * v * (1 - u^2.168 / (1 + 0.007 * u^2.168))


param_bounds_symbolic_net_l1_reg_0.1_repeat_34_lr_0.001
0.010 * u * u
-0.629 * u^1.361 / (1 + 0.014 * u^1